In [ ]:
import ast

def node2state(node, info, level):
    info["level"] = level
    if isinstance(node, ast.AST):
        for name, val in ast.iter_fields(node):
            if "ast" in str(val):
                info[name] = None
            else:
                info[name] = val
        for name, val in ast.iter_fields(node):
            if name not in ('left', 'right'):
                node2state(val, info, level)
        info["type"] = node.__class__.__name__


def traverse(node, states, level=0):
    info = {}
    node2state(node, info, level)
    states.append(info)
    for field, value in ast.iter_fields(node):
        if isinstance(value, list):
            for item in value:
                if isinstance(item, ast.AST):
                    traverse(item, states, level=level+1)
        elif isinstance(value, ast.AST):
            traverse(value, states, level=level+1)


def parse(code):
    return ast.parse(code)

def classify_state(states, label):
    for state in states:
        state["label"] = label


def code2states(code,label):
    states = []
    try:
        tree = parse(code)
    except:
        print(code)
    traverse(tree, states)
    classify_state(states, label)
    return states

def create_states(code,labels):
    states = []
    code_parts = code.split("#\r\n")
    label_parts = labels.split(",")
    for i in range(len(code_parts)):
        s = code2states(code_parts[i],label_parts[i])
        states = [*states,*s]
    return states




In [ ]:
create_states("class MyClass:\r\n    while True:\r\n        return False #Noncompliant","GOOD")